In [3]:
#REMOVE
# import onnx

# from onnx_tf.backend import prepare

# onnx_model = onnx.load("reusr.onnx")  # load onnx model
# tf_rep = prepare(onnx_model)  # prepare tf representation
# tf_rep.export_graph("reusr-tf.pb")  # export the model

# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()

ModuleNotFoundError: No module named 'tensorflow.python.ops.standard_ops'

# Import Libraries

In [1]:
import os
import numpy as np
from pathlib import Path
import re
import tensorflow as tf

# from dotenv import load_dotenv
# load_dotenv()

# Configure, Load Dataset 

Dataset of **469** images scraped from google belonging to 6 categories: **cardboard, glass bottles, plastic containers, paper, glass jars and plastic bottles**

Train dataset: 378 images
Test dataset: 91 images

![](https://snap-n-reuse.netlify.app/assets/img/landing/f3.png)

In [2]:
data_dir = 'data'
TRAIN_DATASET_PATH = data_dir + '/train'
VALIDATION_DATASET_PATH = data_dir + '/test'

In [3]:
TRAIN_DATASET_PATH

'data/train'

In [4]:
BATCH_SIZE = 8
N_CLASSES = 6
TOTAL_EPOCHS = 100

In [5]:
# Generators
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    data_format='channels_last',
    rescale=1. / 255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

train_batches = train_generator.flow_from_directory(
    batch_size=BATCH_SIZE,
    directory=TRAIN_DATASET_PATH,
    target_size=[224, 224],
    class_mode='categorical'
)

val_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    data_format='channels_last',
    rescale=1. / 255
)

val_batches = train_generator.flow_from_directory(
    batch_size=BATCH_SIZE,
    directory=VALIDATION_DATASET_PATH,
    target_size=[224, 224],
    class_mode='categorical'
)

Found 378 images belonging to 6 classes.
Found 91 images belonging to 6 classes.


# Model Config

In [6]:
# Model
kernel_initializer = tf.keras.initializers.glorot_uniform(seed=1337)
trained_model = tf.keras.applications.mobilenet_v2.MobileNetV2(
                      include_top=False,
                      weights='imagenet',
                      classes=N_CLASSES,
                      alpha=0.5,
                      input_shape=[224, 224, 3],
                      pooling='max')
output = tf.keras.layers.Dense(N_CLASSES, activation='softmax', kernel_initializer=kernel_initializer)(trained_model.output)
model = tf.keras.Model(inputs=trained_model.input, outputs=output)

In [7]:
# Callback to save weights, based on val_acc
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
  './checkpoints/{epoch:02d}_{val_accuracy:.4f}.h5',
  save_weights_only=False,
  verbose=1,
  monitor='val_accuracy',
  save_best_only=True,
#     save_freq='epoch',
  mode='max'
)

# Callbackto plot data on TensorBoard
tensorboard_callback = tf.keras.callbacks.TensorBoard(
  log_dir='./logs/reusr',
  histogram_freq=0,
#   batch_size=BATCH_SIZE
)

# Callback to reduce learning rate after plateaus
reduce_lr_callback = tf.keras.callbacks.ReduceLROnPlateau(
  monitor='val_accuracy',
  factor=0.5,
  patience=4,
  min_lr=1e-6
)

early_stopping_callback = tf.keras.callbacks.EarlyStopping(
  monitor='val_accuracy',
  patience=20,
  mode='max',
)

In [8]:
TRAIN_DATASET_SIZE = len(train_batches)
VAL_DATASET_SIZE   = len(val_batches)

# Weighted losses for class equilibrium
unique, counts = np.unique(train_batches.classes, return_counts=True)
class_weigths = dict(zip(unique, np.true_divide(counts.sum(), N_CLASSES*counts)))


if Path('./checkpoints/').exists():
  epoch_number_array = []
  val_accuracy_array = []
  file_name_array = []
  for file in os.listdir('./checkpoints/'):
    epoch, val_acc = re.search(r'(\d\d)_(\d\.\d{4})\.h5', file).group(1,2)
    epoch_number_array.append(int(epoch))
    val_accuracy_array.append(float(val_acc))
    file_name_array.append(file)

  if len(val_accuracy_array) == 0:
    INITIAL_EPOCH = 0
  else:
    highest_acc = val_accuracy_array.index(max(val_accuracy_array))
    INITIAL_EPOCH = epoch_number_array[highest_acc]
    model_checkpoint_callback.best = val_accuracy_array[highest_acc]
    model.load_weights('./checkpoints/'+file_name_array[highest_acc])
else:
  os.makedirs('./checkpoints/')
  INITIAL_EPOCH = 0

# Run Model

In [9]:
# Prepare model to run
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss = 'categorical_crossentropy',
              metrics = ['accuracy']
              )

In [10]:
#Starts training the model
model.fit(train_batches,
        epochs=TOTAL_EPOCHS,
        verbose=1,
        steps_per_epoch=TRAIN_DATASET_SIZE,
        validation_data=val_batches,
        validation_steps=VAL_DATASET_SIZE,
#         validation_split=0.1,
        initial_epoch=INITIAL_EPOCH,
        class_weight=class_weigths,
#         use_multiprocessing=True,
        workers=4,
        callbacks=[model_checkpoint_callback, tensorboard_callback, reduce_lr_callback, early_stopping_callback]
        )

Epoch 1/100
48/48 [==============================] - ETA: 0s - loss: 4.0101 - accuracy: 0.4471
Epoch 00001: val_accuracy improved from -inf to 0.25275, saving model to ./checkpoints/01_0.2527.h5
48/48 [==============================] - 49s 1s/step - loss: 4.0101 - accuracy: 0.4471 - val_loss: 5.4826 - val_accuracy: 0.2527 - lr: 0.0010
Epoch 2/100
48/48 [==============================] - ETA: 0s - loss: 3.7789 - accuracy: 0.5212
Epoch 00002: val_accuracy did not improve from 0.25275
48/48 [==============================] - 40s 843ms/step - loss: 3.7789 - accuracy: 0.5212 - val_loss: 6.0274 - val_accuracy: 0.1538 - lr: 0.0010
Epoch 3/100
48/48 [==============================] - ETA: 0s - loss: 3.0154 - accuracy: 0.5476
Epoch 00003: val_accuracy did not improve from 0.25275
48/48 [==============================] - 35s 739ms/step - loss: 3.0154 - accuracy: 0.5476 - val_loss: 7.9196 - val_accuracy: 0.2088 - lr: 0.0010
Epoch 4/100
48/48 [==============================] - ETA: 0s - loss: 2.82

Epoch 27/100
48/48 [==============================] - ETA: 0s - loss: 0.1434 - accuracy: 0.9497
Epoch 00027: val_accuracy did not improve from 0.73626
48/48 [==============================] - 34s 713ms/step - loss: 0.1434 - accuracy: 0.9497 - val_loss: 1.2916 - val_accuracy: 0.6923 - lr: 2.5000e-04
Epoch 28/100
48/48 [==============================] - ETA: 0s - loss: 0.2020 - accuracy: 0.9339
Epoch 00028: val_accuracy did not improve from 0.73626
48/48 [==============================] - 35s 720ms/step - loss: 0.2020 - accuracy: 0.9339 - val_loss: 1.4804 - val_accuracy: 0.6593 - lr: 1.2500e-04
Epoch 29/100
48/48 [==============================] - ETA: 0s - loss: 0.1996 - accuracy: 0.9444
Epoch 00029: val_accuracy did not improve from 0.73626
48/48 [==============================] - 34s 718ms/step - loss: 0.1996 - accuracy: 0.9444 - val_loss: 1.5267 - val_accuracy: 0.6813 - lr: 1.2500e-04
Epoch 30/100
48/48 [==============================] - ETA: 0s - loss: 0.1376 - accuracy: 0.9365
Epoc

48/48 [==============================] - ETA: 0s - loss: 0.0613 - accuracy: 0.9762
Epoch 00080: val_accuracy did not improve from 0.90110
48/48 [==============================] - 34s 704ms/step - loss: 0.0613 - accuracy: 0.9762 - val_loss: 0.5815 - val_accuracy: 0.8681 - lr: 1.0000e-06
Epoch 81/100
48/48 [==============================] - ETA: 0s - loss: 0.0687 - accuracy: 0.9709
Epoch 00081: val_accuracy did not improve from 0.90110
48/48 [==============================] - 32s 659ms/step - loss: 0.0687 - accuracy: 0.9709 - val_loss: 0.7098 - val_accuracy: 0.8352 - lr: 1.0000e-06
Epoch 82/100
48/48 [==============================] - ETA: 0s - loss: 0.0542 - accuracy: 0.9815
Epoch 00082: val_accuracy did not improve from 0.90110
48/48 [==============================] - 32s 675ms/step - loss: 0.0542 - accuracy: 0.9815 - val_loss: 0.4342 - val_accuracy: 0.8901 - lr: 1.0000e-06
Epoch 83/100
48/48 [==============================] - ETA: 0s - loss: 0.0561 - accuracy: 0.9788
Epoch 00083: val_

~90% validation accuracy achieved. Trained tensorflow model saved in h5 format

In [11]:
MODEL_PATH = 'models/reusr-keras.h5'
tf.keras.models.save_model(
model,
MODEL_PATH,
overwrite=True,
include_optimizer=True,
save_format=None
)

# TensorflowJS model generator code

converting tensorflow model to tfjs model for easy use in web app

In [12]:
!tensorflowjs_converter \
--input_format=keras \
models/reusr-keras.h5 \
models/reusr-tfjs

2020-11-21 06:33:02.665413: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library cudart64_101.dll


2 files: model.json and shard files are generated from the above code